# Baseline
python: 3.8.*

use ```Ctrl + ]``` to collapse all section :)

Download our starter pack (3~5 min)

In [ ]:
!gdown 1Xq2Fv6UGA1pc25pF0qwEc_l7Fa5jPP6p

In [2]:
!gdown --folder 1T6jpOtdf_i6XNYA6F_lqU4mRRh1xYPcl
!mv baseline/* ./

Retrieving folder 15cJ_K2Tm95UkHgnvvPdMqGQy0_At6lRI checkpoints
Retrieving folder 1jIWaLk4VLXEEm7au12mH80bKqFS3-ZKk claim_verification
Retrieving folder 1DVfRginiE0thVjKELz7DCsEMYltRaTDG e20_bs32_7e-05_top5
Processing file 1AGMfElEghEnYWLYU0YURZrBzmskYuovm val_acc=0.4259_model.750.pt
Retrieving folder 19-pRgA1elB6U2UklW6DmUUm9R4qraAu8 sent_retrieval
Retrieving folder 15zhuj7t3s5vG4JtFS_PNRTtE5l9mv0nZ e1_bs64_2e-05_neg0.03_top5
Processing file 1PBBfgu9lV8_hHdmZRT99bAk_Zl51MZcr model.50.pt
Retrieving folder 1r742uPeVnqUm04qUEZpzzpxXZFbj4gNd data
Processing file 1hBMys30E2Tw-QCt8FDKKnu-eF_G1tGio dev_doc5sent5.jsonl
Processing file 1iHSNpooXDLurizn9sxUbY1Fpk-h9La3i dev_doc5sent5.pkl
Processing file 1a0e__D64L8CWhZsvuYZZuXiOZ7a99Bnj hanlp_con_results.pkl
Processing file 1BFeoxCm5n0fauW9rbNqy6Se9dK9RnbEJ hanlp_con_test_results.pkl
Processing file 1rTvOEQAZG7Hs5aVlTvpnZSW89JWNTUzK public_test.jsonl
Processing file 1TbIsMs71WZP2kRpgnn2U383mPavu1G-b public_train.jsonl
Processing file 1PDGOk2CCw

Retrieving folder list
Retrieving folder list completed
Building directory structure

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1AGMfElEghEnYWLYU0YURZrBzmskYuovm 

Download ended unsuccessfully
'mv' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [ ]:
!gdown --folder 1hnVYEgN-gYzFCeBZo8cbKjGLBP-YTnTW

In [1]:
%pip install -r requirements.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 138 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 120.6 MB/s eta 0:00:01     |███▌                            | 1.1 MB 120.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 126.9 MB/s eta 0:00:01
     |████████████████████████████████| 1977.9 MB 2.8 kB/s s eta 0:00:01   |███▊                            | 231.9 MB 74.4 MB/s eta 0:00:24     |█████████████▌                  | 837.8 MB 74.0 MB/s eta 0:00:16     |████████████████                | 992.8 MB 22.9 MB/s eta 0:00:43     |███████████████████             | 1173.7 MB 22.9 MB/s eta 0:00:36     |███████████████████▌            | 1207.3 MB 62.1 MB/s eta 0:00:13�███████▊            | 1220.7 MB 62.1 MB/s eta 0:00:13     |████████████████████▍           | 1259.6 MB 21.1 MB/s eta 0:00:35     |█████████████████████▌         

## PART 1. Document retrieval

Prepare the environment and import all library we need

In [1]:
import json
import pickle
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Set, Tuple, Union
from functools import partial

# 3rd party libs
import hanlp
import opencc
import pandas as pd
from hanlp.components.pipeline import Pipeline
from pandarallel import pandarallel

# our own libs
from utils import load_json
from hw3_utils import jsonl_dir_to_df

pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)

In [2]:
from TCSP import read_stopwords_list

stopwords = read_stopwords_list()

Preload the data.

In [3]:
TRAIN_DATA = load_json("data/public_train.jsonl")
TEST_DATA = load_json("data/public_test.jsonl")
CONVERTER_T2S = opencc.OpenCC("t2s.json")
CONVERTER_S2T = opencc.OpenCC("s2t.json")

Data class for type hinting

In [4]:
@dataclass
class Claim:
    data: str

@dataclass
class AnnotationID:
    id: int

@dataclass
class EvidenceID:
    id: int

@dataclass
class PageTitle:
    title: str

@dataclass
class SentenceID:
    id: int

@dataclass
class Evidence:
    data: List[List[Tuple[AnnotationID, EvidenceID, PageTitle, SentenceID]]]

### Helper function

For the sake of consistency, we convert traditional to simplified Chinese first before converting it back to traditional Chinese.  This is due to some errors occuring when converting traditional to traditional Chinese.

In [5]:
def do_st_corrections(text: str) -> str:
    simplified = CONVERTER_T2S.convert(text)

    return CONVERTER_S2T.convert(simplified)

We use constituency parsing to separate part of speeches or so called constituent to extract noun phrases.  In the later stages, we will use the noun phrases as the query to search for relevant documents.  

In [6]:
def get_nps_hanlp(
    predictor: Pipeline,
    d: Dict[str, Union[int, Claim, Evidence]],
) -> List[str]:
    claim = d["claim"]
    tree = predictor(claim)["con"]
    nps = [
        do_st_corrections("".join(subtree.leaves()))
        for subtree in tree.subtrees(lambda t: t.label() == "NP")
    ]

    return nps

Precision refers to how many related documents are retrieved.  Recall refers to how many relevant documents are retrieved.  

In [5]:
def calculate_precision(
    data: List[Dict[str, Union[int, Claim, Evidence]]],
    predictions: pd.Series,
) -> float:
    precision = 0
    count = 0

    for i, d in enumerate(data):
        if d["label"] == "NOT ENOUGH INFO":
            continue

        # Extract all ground truth of titles of the wikipedia pages
        # evidence[2] refers to the title of the wikipedia page
        gt_pages = set([
            evidence[2]
            for evidence_set in d["evidence"]
            for evidence in evidence_set
        ])

        predicted_pages = predictions.iloc[i]
        hits = predicted_pages.intersection(gt_pages)
        if len(predicted_pages) != 0:
            precision += len(hits) / len(predicted_pages)

        count += 1

    # Macro precision
    print(f"Precision: {precision / count}")
    return precision / count


def calculate_recall(
    data: List[Dict[str, Union[int, Claim, Evidence]]],
    predictions: pd.Series,
) -> float:
    recall = 0
    count = 0

    for i, d in enumerate(data):
        if d["label"] == "NOT ENOUGH INFO":
            continue

        gt_pages = set([
            evidence[2]
            for evidence_set in d["evidence"]
            for evidence in evidence_set
        ])
        predicted_pages = predictions.iloc[i]
        hits = predicted_pages.intersection(gt_pages)
        recall += len(hits) / len(gt_pages)
        count += 1

    print(f"Recall: {recall / count}")
    return recall / count

In [6]:
def calculate_f1(precision: float, recall: float) -> float:
    return 2*(precision*recall)/(precision+recall)

The default amount of documents retrieved is at most five documents.  This `num_pred_doc` can be adjusted based on your objective.  Save data in jsonl format.

In [7]:
def save_doc(
    data: List[Dict[str, Union[int, Claim, Evidence]]],
    predictions: pd.Series,
    mode: str = "train",
    suffix: str = "",
    num_pred_doc: int = 5,
) -> None:
    with open(
        f"data/{mode}_doc{num_pred_doc}{suffix}.jsonl",
        "w",
        encoding="utf8",
    ) as f:
        for i, d in enumerate(data):
            d["predicted_pages"] = list(predictions.iloc[i])
            f.write(json.dumps(d, ensure_ascii=False) + "\n")

### Tfidf Init

In [8]:
def tokenize(text: str, stopwords: list) -> str:
    import jieba
    tokens = jieba.cut(text)

    return " ".join([w for w in tokens if w not in stopwords])

In [9]:
wiki_path = "data/wiki-pages"
min_wiki_length = 15
num_of_samples = 500
topk = 15
min_df = 1
max_df = 0.8
use_idf = True
sublinear_tf = True

# Set up the experiment name for logging
exp_name = (
    f"len{min_wiki_length}_top{topk}_min_df={min_df}_"
    + f"max_df={max_df}_{num_of_samples}s"
)
if sublinear_tf:
    exp_name = "sublinearTF_" + exp_name
if not use_idf:
    exp_name = "no_idf_" + exp_name

In [10]:
wiki_cache = "wiki"
target_column = "text"

wiki_cache_path = Path(f"data/{wiki_cache}.pkl")
if wiki_cache_path.exists():
    wiki_pages = pd.read_pickle(wiki_cache_path)
else:
    # You need to download `wiki-pages.zip` from the AICUP website
    wiki_pages = jsonl_dir_to_df(wiki_path)
    # wiki_pages are combined into one dataframe, so we need to reset the index
    wiki_pages = wiki_pages.reset_index(drop=True)

    # tokenize the text and keep the result in a new column `processed_text`
    wiki_pages["processed_text"] = wiki_pages[target_column].parallel_apply(
        partial(tokenize, stopwords=stopwords)
    )
    # save the result to a pickle file
    wiki_pages.to_pickle(wiki_cache_path, protocol=4)

In [11]:
wiki_pages = wiki_pages[
    wiki_pages['processed_text'].str.len() > min_wiki_length
]
corpus = wiki_pages["processed_text"].tolist()

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy

# vectorizer = TfidfVectorizer(
#     min_df=min_df,
#     max_df=max_df,
#     use_idf=use_idf,
#     sublinear_tf=sublinear_tf,
#     dtype=np.float64,
# )

In [13]:
import gensim.models
w2vmodel = gensim.models.Word2Vec.load("models/w2v.zh.300/word2vec.model")
w2v = dict(zip(w2vmodel.wv.index_to_key, w2vmodel.wv.vectors))

In [14]:
from collections import defaultdict
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec, size=300):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = size
    
    def fit(self, X):
        tfidf = TfidfVectorizer(
            min_df=1,
            max_df=0.8,
            use_idf=True,
            sublinear_tf=True,
            dtype=np.float64,
            analyzer=lambda x: x
        )
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [15]:
vectorizer = TfidfEmbeddingVectorizer(w2v)

In [16]:
X = vectorizer.fit(corpus).transform(corpus)

In [26]:
# Save matrix
from scipy import sparse
sparse_mtx_name = f"data/X.npz"
if Path(sparse_mtx_name).exists:
    sparse.save_npz(sparse_mtx_name, X)
else:
    sparse.load_npz(sparse_mtx_name)

AttributeError: 'numpy.ndarray' object has no attribute 'format'

### Main function for document retrieval

In [19]:
def get_pred_pages(
        series_data: pd.Series, 
        ) -> Set[Dict[int, str]]:
    import wikipedia
    import re
    import opencc
    import pandas as pd

    from TCSP import read_stopwords_list
    stopwords = read_stopwords_list()

    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    wikipedia.set_lang("zh")
    CONVERTER_T2S = opencc.OpenCC("t2s.json")
    CONVERTER_S2T = opencc.OpenCC("s2t.json")
    
    def do_st_corrections(text: str) -> str:
        simplified = CONVERTER_T2S.convert(text)
        return CONVERTER_S2T.convert(simplified)

    results = []
    tmp_muji = []
    # wiki_page: its index showned in claim
    mapping = {}
    claim = series_data["claim"]
    nps = series_data["hanlp_results"]
    first_wiki_term = []
    repeated_mention = []
    quote_search = []

    def clean_claim(claim) -> str:     # Clean claim function because hanlp has error when conducting cons
        def multiple_replacer(*kv):
            replace_dict = dict(kv)
            replace_func = lambda match: replace_dict[match.group(0)]
            pattern = re.compile("|".join([re.escape(k) for k, v in kv]), re.M)
            return lambda string: pattern.sub(replace_func, string) 
        def multiple_replace(string, *kv):
            return multiple_replacer(*replace_dict)(claim)

        replace_dict = (" ", ""), ("牠", "它"), ("（", "("), ("）", ")"), ("，", ","), ("、", ","), ("群", "羣"), ("“", "\""), ("”", "\""), ("「", "“"), ("」", "”")
        claim = multiple_replace(claim, *replace_dict)
        claim = claim.lower()
        return claim

    claim = clean_claim(claim)

    def post_processing(np, page, loc):
        page = do_st_corrections(page)
        page = page.replace(" ", "_")
        page = page.replace("-", "")
        search_pos = claim.find(np)
        if search_pos != -1:
            if page in results:
                repeated_mention.append(page)
                # results.insert(0, results.pop(results.index(page)))     # Fresh page to front if it was mention before
            else:
                results.append(page)
            if loc == 0:
                pass
                # print(f"Add: {page}, at page direct search, np={np}")
            elif loc == 1:
                pass
                # print(f"Add: {page}, at match, new term={np}")
            mapping[page] = search_pos
            tmp_muji.append(np)

    def if_page_exists(page: str) -> bool:
        import requests
        url_base = "https://zh.wikipedia.org/wiki/"
        new_url = [url_base + page, url_base + page.upper()]
        for url in new_url:
            r = requests.head(url)
            if r.status_code == 200:
                return True
            else:
                continue
        return False
    
    def clean_time_format(np: str):
        if (matched := re.search(r"\d+年", np)) != None:
            return True
        if (matched := re.search(r"\d+月\d+日", np)) != None:
            return True
        if (matched := re.search(r"\d+小時", np)) != None:
            return True
        if (matched := re.search(r"\d+天", np)) != None:
            return True
        if (matched := re.search(r"\d+世紀", np)) != None:
            return True
        if (matched := re.search(r"\d+年代", np)) != None:
            return True
        return False
    
    def tokenize(text: str, stopwords: list) -> str:
        import jieba
        """This function performs Chinese word segmentation and removes stopwords.

        Args:
            text (str): claim or wikipedia article
            stopwords (list): common words that contribute little to the meaning of a sentence

        Returns:
            str: word segments separated by space (e.g. "我 喜歡 吃 蘋果")
        """

        tokens = jieba.cut(text)

        return " ".join([w for w in tokens if w not in stopwords])

    for i, np in enumerate(nps):
        # print(f"searching {np}")
        quote_dup = False
        if np in stopwords:         # 如果包含停用詞
            continue
        if clean_time_format(np):   # 如果包含時間
            continue
        
        # Ignore parsing among quotation mark, for example, if《仲夏夜之夢》exists, ignore「仲夏夜」and「夢」
        for search in quote_search:
            if search.find(np) != -1:
                quote_dup = True
        if quote_dup == True:
            continue

        # Delete Bookname Mark, Quote Mark
        np_no_quote = re.sub(r"《|》|〈|〉|【|】|「|」|『|』|（|）", "", np)
        if np != np_no_quote:
            quote_search.append(np_no_quote)
            np = np_no_quote

        # Simplified Traditional Chinese Correction
        wiki_search_results = [
            do_st_corrections(w) for w in wikipedia.search(np)
        ]

        # Directly Search by Redirection
            # Check if a page exists
        if (if_page_exists(np)):
            try:
                page = do_st_corrections(wikipedia.page(title=np).title)
                if page == np:
                    # print(f"Found, np={np}, page={page}")
                    post_processing(np=np, page=page, loc=0)
                else:
                    # print(f"Redirect, np={np}, page={page}")
                    post_processing(np=np, page=page, loc=0)
            except wikipedia.DisambiguationError as diserr:
                page = do_st_corrections(wikipedia.search(np)[0])
                if page == np:
                    # print(f"Disambig, np={np}, page={page}")
                    post_processing(np=np, page=page, loc=0)
            except wikipedia.PageError as pageerr:
                pass

        # Remove the wiki page's description in brackets
        wiki_set = [re.sub(r"\s\(\S+\)", "", w) for w in wiki_search_results]
        wiki_df = pd.DataFrame({
            "wiki_set": wiki_set,
            "wiki_results": wiki_search_results
        })

        # Elements in wiki_set --> index
        # Extracting only the first element is one way to avoid extracting
        # too many of the similar wiki pages
        grouped_df = wiki_df.groupby("wiki_set", sort=False).first()
        candidates = grouped_df["wiki_results"].tolist()
        # muji refers to wiki_set
        muji = grouped_df.index.tolist()

        for prefix, term in zip(muji, candidates):
            if prefix not in tmp_muji:  #忽略掉括號，如果括號有重複的話。假設如果有" 1 (數字)", 則"1 (符號)" 會被忽略
                matched = False

                # Take at least one term from the first noun phrase
                if i == 0:
                    first_wiki_term.append(term)

                # try:
                #     print(term)
                #     term_idx = wiki_pages.index[wiki_pages['id'] == do_st_corrections(term.replace(" ", "_").replace("-", ""))].tolist()[0]
                #     processed_tokens = wiki_pages['processed_text'][term_idx]
                #     processed_text_vector = vectorizer.transform([processed_tokens])
                #     sim_score = cosine_similarity(processed_text_vector, claim_vector)[0][0]
                #     if sim_score > 0.25: # 0.25 is hyperparam
                #         score_mapping[term] = sim_score
                #         print(sim_score, term)
                # except IndexError:
                #     pass
                # except wikipedia.DisambiguationError:
                #     pass
                # except wikipedia.PageError:
                #     pass

                # Walrus operator :=
                # https://docs.python.org/3/whatsnew/3.8.html#assignment-expressions
                # Through these filters, we are trying to figure out if the term
                # is within the claim
                if (((new_term := term) in claim) or
                    ((new_term := term) in claim.replace(" ", "")) or
                    ((new_term := term.replace("·", "")) in claim) or                                   # 過濾人名
                    ((new_term := re.sub(r"\s\(\S+\)", "", term)) in claim) or                          # 過濾空格 / 消歧義
                    ((new_term := term.replace("(", "").replace(")", "").split()[0]) in claim and       # 消歧義與括號內皆有在裡面
                     (new_term := term.replace("(", "").replace(")", "").split()[1]) in claim) or
                    ((new_term := term.replace("-", " ")) in claim) or                                  # 過濾槓號
                    ((new_term := term.lower()) in claim) or                                            # 過濾大小寫
                    ((new_term := term.lower().replace("-", "")) in claim) or                           # 過濾大小寫及槓號
                    ((new_term := re.sub(r"\s\(\S+\)", "", term.lower().replace("-", ""))) in claim)    # 過濾大小寫、槓號及消歧義
                    ):
                    matched = True
                    # print(new_term, term)

                # 人名匹配
                elif "·" in term:
                    splitted = term.split("·")
                    if "·" not in claim:        # 要求claim顯示的不為全名，不然都需要全名
                        for split in splitted:
                            if (new_term := split) in claim:
                                matched = True
                                break

                if matched:
                    post_processing(np=new_term, page=term, loc=1)

    # score_results = sorted(score_mapping, key=score_mapping.get)[:-5]

    # 8 is a hyperparameter
    if len(results) > 8:
        assert -1 not in mapping.values()
        # print("長度大於8", results)

        results = repeated_mention + sorted(mapping, key=mapping.get)[:8]
        results = list(set(results))            # remove duplicates
        # print("排序後", results)
    if len(results) < 1:
        results = first_wiki_term
        # print("第一搜尋結果", results)
    
    print(results)
    return set(results)

In [18]:
def get_pred_pages_search(
        series_data: pd.Series, 
        ) -> Set[Dict[int, str]]:
    import wikipedia
    import re
    import opencc
    import pandas as pd

    import numpy as np

    wikipedia.set_lang("zh")
    CONVERTER_T2S = opencc.OpenCC("t2s.json")
    CONVERTER_S2T = opencc.OpenCC("s2t.json")
    
    def do_st_corrections(text: str) -> str:
        simplified = CONVERTER_T2S.convert(text)
        return CONVERTER_S2T.convert(simplified)

    results = []
    tmp_muji = []
    nps = series_data["hanlp_results"]

    def post_processing(page):
        page = do_st_corrections(page)
        page = page.replace(" ", "_")
        page = page.replace("-", "")

    for i, np in enumerate(nps):
        # print(f"searching {np}")

        # Simplified Traditional Chinese Correction
        wiki_search_results = [
            do_st_corrections(w) for w in wikipedia.search(np)
        ]

        # Remove the wiki page's description in brackets
        wiki_set = [re.sub(r"\s\(\S+\)", "", w) for w in wiki_search_results]
        wiki_df = pd.DataFrame({
            "wiki_set": wiki_set,
            "wiki_results": wiki_search_results
        })

        # Elements in wiki_set --> index
        # Extracting only the first element is one way to avoid extracting
        # too many of the similar wiki pages
        grouped_df = wiki_df.groupby("wiki_set", sort=False).first()
        candidates = grouped_df["wiki_results"].tolist()
        # muji refers to wiki_set
        muji = grouped_df.index.tolist()

        for prefix, term in zip(muji, candidates):
            if prefix not in tmp_muji:  #忽略掉括號，如果括號有重複的話。假設如果有" 1 (數字)", 則"1 (符號)" 會被忽略
                post_processing(term)
                results.append(term)

    results = list(set(results))            # remove duplicates
    print(results)
    return set(results)

In [17]:
def get_pred_pages_tfidf(
    series_data: pd.Series, 
    tokenizing_method: callable,
    vectorizer: TfidfVectorizer,
    tf_idf_matrix: scipy.sparse.csr_matrix,
    wiki_pages: pd.DataFrame,
    topk: int
) -> set:
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

    claim = series_data["claim"]
    search_list = series_data["predicted_pages"]
    results = []
    mapping = {}

    tokens = tokenizing_method(claim)
    claim_vector = vectorizer.transform([tokens])
    for search_id in search_list:
        search_tokens = wiki_pages.loc[wiki_pages['id'] == search_id]
        if search_tokens.empty:
            continue
        search_processed_text = search_tokens["processed_text"]
        search_vector = vectorizer.transform(search_processed_text)
        sim_scores = cosine_similarity(search_vector, claim_vector)
        sim_scores = sim_scores[0][0]
        if sim_scores > 0.8:
            mapping[search_id] = sim_scores
            # print(sim_scores, search_id)

    # print(mapping)
    results = sorted(mapping, key=mapping.get, reverse=True)[:5]
    print(results)
    return set(results)


### Step 1. Get noun phrases from hanlp consituency parsing tree

Setup [HanLP](https://github.com/hankcs/HanLP) predictor (1 min)

In [37]:
predictor = (hanlp.pipeline().append(
    hanlp.load("FINE_ELECTRA_SMALL_ZH"),
    output_key="tok",
).append(
    hanlp.load("CTB9_CON_ELECTRA_SMALL"),
    output_key="con",
    input_key="tok",
))

We will skip this process which for creating parsing tree when demo on class

In [38]:
hanlp_file = f"data/hanlp_con_results.pkl"
if Path(hanlp_file).exists():
    with open(hanlp_file, "rb") as f:
        hanlp_results = pickle.load(f)
else:
    hanlp_results = [get_nps_hanlp(predictor, d) for d in TRAIN_DATA]
    with open(hanlp_file, "wb") as f:
        pickle.dump(hanlp_results, f)

Get pages via wiki online api

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [22]:
doc_path = f"data/train_doc5.jsonl"
doc_path_aicup = f"data/train_doc5_aicup.jsonl"
doc_path_search = f"data/train_doc5_search.jsonl"
doc_path_tfidf = f"data/train_doc5_tfidf.jsonl"

if Path(doc_path).exists():
    with open(doc_path_aicup, "r", encoding="utf8") as f:
        predicted_results = pd.Series([
            set(json.loads(line)["predicted_pages"])
            for line in f
        ])
else:
    if Path(doc_path_search).exists():
        with open(doc_path_search, "r", encoding="utf8") as f:
            predicted_results_search = pd.Series([
                set(json.loads(line)["predicted_pages"])
                for line in f
            ], name="search")
    else:
        pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)
        train_df = pd.DataFrame(TRAIN_DATA)
        train_df.loc[:, "hanlp_results"] = hanlp_results
        # predicted_results = train_df.progress_apply(get_pred_pages, axis=1)
        predicted_results_search = train_df.parallel_apply(
            get_pred_pages_search, axis=1)
        save_doc(TRAIN_DATA, predicted_results_search, mode="train", suffix="_search")

    if Path(doc_path_aicup).exists():
        with open(doc_path_aicup, "r", encoding="utf8") as f:
            predicted_results_aicup = pd.Series([
                set(json.loads(line)["predicted_pages"])
                for line in f
            ], name="aicup")
    else:
        pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)
        train_df = pd.DataFrame(TRAIN_DATA)
        train_df.loc[:, "hanlp_results"] = hanlp_results
        # predicted_results = train_df.progress_apply(get_pred_pages, axis=1)
        predicted_results_aicup = train_df.parallel_apply(
            get_pred_pages, axis=1)
        save_doc(TRAIN_DATA, predicted_results_aicup, mode="train", suffix="_aicup")

    if Path(doc_path_tfidf).exists():
        with open(doc_path_tfidf, "r", encoding="utf8") as f:
            predicted_results_tfidf = pd.Series([
                set(json.loads(line)["predicted_pages"])
                for line in f
            ], name="tfidf")
    else:
        pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=3)
        TRAIN_DATA_SEARCH = load_json(doc_path_search)
        train_df_search = pd.DataFrame(TRAIN_DATA_SEARCH)
        predicted_results_tfidf = train_df_search.parallel_apply(
            partial(
                get_pred_pages_tfidf,
                tokenizing_method=partial(tokenize, stopwords=stopwords),
                vectorizer=vectorizer,
                tf_idf_matrix=X,
                wiki_pages=wiki_pages,
                topk=topk,
            ), axis=1)
        save_doc(TRAIN_DATA, predicted_results_tfidf, mode="train", suffix="_tfidf")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.391 seconds.
Prefix dict has been built successfully.


['緹坦妮雅', '奧伯隆']


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.372 seconds.
Prefix dict has been built successfully.


[]
['漂泊信天翁', '桑氏遠洋鳥', '信天翁科', '黑腳信天翁', '風神翼龍屬']
[]


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.358 seconds.
Prefix dict has been built successfully.


[]
['麗山']
['嶺南師範學院', '中華民國大陸時期國立大專校院列表', '韓山師範學院', '國立師範學院', '華南師範大學附屬中學']
['黎巴嫩', '世界語言', '阿拉伯語變體', '英語國家和地區列表', '穆斯林']
['旁遮普地區', '旁遮普人']
['F.I.R.飛兒樂團', '椅子樂團', '回聲樂團', 'TOP1男子漢', '酷愛樂團']
['馬克思學說研究會', '馬克思主義思想流派', '馬克思主義政治經濟學', '西方馬克思主義', '對馬克思主義的批評']
['惠靈頓', '紐西蘭人口', '博物館']
['香港國際機場', '希思羅機場', '吉隆坡國際機場', '啓德機場', '機場']
['吉林大學附屬中學', '吉林大學', '吉林農業大學', '中華人民共和國', '皓月集團']
[]
['中國東北地區', '中國大陸人口', '育空市鎮列表', '內蒙古自治區各盟市面積列表', '內蒙古各市盟地區生產總值列表']
['海王星的衛星', '太陽系天體列表', '太陽系', '行星定義', '木星的衛星']
['黨委書記和校長列入中央管理的高校', '地方所屬高校', '高等學校', '中國大陸高等學校列表', '技術型高級中等學校']
[]
['南京師範大學附屬中學', '江蘇省蘇州實驗中學', '南京大學附屬中學', '江蘇省政府']
[]
['釋迦牟尼', '釋迦族', '喬達摩', '佛教', '牟尼']
['植酸', '脂類', '磷酸肌醇3-激酶', '磷脂酶C']
[]
['毒魚豆']
['環境科學', '環境保護', '環境資源管理']
['惠靈頓', '紐西蘭人口']
[]
['社會問題']
['軟件開發', '軟件開發工具包', '軟件項目管理', '軟件開發工具', '軟件測試']
[]
['康泰旅行社', '波士頓']
['蛛形綱', '香港純蛛', '蜘蛛', '鷲蛛總科', '櫛足蛛科']
['馬克思學說研究會', '馬克思主義思想流派', '西方馬克思主義', '後馬克思主義', '新馬克思主義']
[]
[]
['國立臺灣大學應用力學研究所', '鮑亦興', '中央研究院', '研究所', '吳政忠']
[]
['德克薩斯州州長', '詹姆斯·諾克斯·波爾克']
['狻猊', '龍生

Save Memory

In [23]:
print("On AICUP Data:")
calculate_precision(TRAIN_DATA, predicted_results_aicup)
calculate_recall(TRAIN_DATA, predicted_results_aicup)
print("On Search Data:")
calculate_precision(TRAIN_DATA, predicted_results_search)
calculate_recall(TRAIN_DATA, predicted_results_search)
print("On Search-TFIDF Data:")
precision = calculate_precision(TRAIN_DATA, predicted_results_tfidf)
recall = calculate_recall(TRAIN_DATA, predicted_results_tfidf)
print(calculate_f1(precision, recall))

On AICUP Data:
Precision: 0.24198516414141477
Recall: 0.8701666666666668
On Search Data:
Precision: 0.01760738788282119
Recall: 0.9074270833333332
On Search-TFIDF Data:
Precision: 0.1764791666666656
Recall: 0.47877083333333337
0.25789569688202013


Merge Two Pandas Series.

In [22]:
def union_result(series_data: pd.Series,) -> set:
    aicup = series_data["aicup"]
    tfidf = series_data["tfidf"]
    print(aicup, tfidf)
    return set(aicup).union(set(tfidf))

In [25]:
results_df = pd.merge(pd.Series([line for line in predicted_results_aicup], name="aicup"), 
                      pd.Series([line for line in predicted_results_tfidf], name="tfidf"), right_index=True, left_index=True)
predicted_results = results_df.apply(union_result, axis=1)
save_doc(TRAIN_DATA, predicted_results, mode="train")

{'天王星', '緹坦妮雅', '仲夏夜之夢', '軌道', '仲夏夜_(羅文專輯)', '磁層', '天衛三'} {'奧伯隆', '緹坦妮雅'}
{'太平洋', '鳥', '南太平洋', '世界', '翼展', '信天翁科', '北冰洋'} {'漂泊信天翁', '桑氏遠洋鳥', '風神翼龍屬', '信天翁科', '黑腳信天翁'}
{'亞洲', '音樂產業', '女性', '阿沁', '男性', '樂團', '詹雯婷', '陳建寧'} {'酷愛樂團', '椅子樂團', 'F.I.R.飛兒樂團', '回聲樂團', 'TOP1男子漢'}
{'小時', '香港國際機場', '機場', '世界', '宇宙', '24小時'} {'香港國際機場', '吉隆坡國際機場', '希思羅機場', '機場', '啓德機場'}
{'中央部屬高校', '高等學校', '中華人民共和國', '歷史', '北京理工大學', '學校', '校長', '黨委書記和校長列入中央管理的高校'} {'高等學校', '技術型高級中等學校', '中國大陸高等學校列表', '地方所屬高校', '黨委書記和校長列入中央管理的高校'}
{'南京大學附屬中學', '江蘇省_(中華民國)', '中學', '南京大學'} {'南京大學附屬中學', '南京師範大學附屬中學', '江蘇省蘇州實驗中學', '江蘇省政府'}
{'印度', '原住民', '西', '萃取', '毒魚豆', '魚'} {'毒魚豆'}
{'軟件', '系統工程', '軟件測試', '軟件開發', '版本控制', '活動'} {'軟件開發工具包', '軟件開發工具', '軟件項目管理', '軟件測試', '軟件開發'}
{'工程力學', '國立臺灣大學', '理論', '力學', '中央研究院', '國立臺灣大學應用力學研究所', '專業人士', '研究生', '院士', '中央研究院院士', '研究所'} {'吳政忠', '中央研究院', '國立臺灣大學應用力學研究所', '鮑亦興', '研究所'}
{'倫琴_(單位)', '威廉·倫琴', 'X射線', '電子'} set()
{'幾何學', '西方', '世紀', '中世紀', '教育', '數學'} {'數學史', '中華人民共和國教育', '博雅教育', '古希臘數學', '數學'}
{'國

### Step 2. Calculate our results

In [34]:
precision = 0.143234
recall = 0.635255
print(calculate_f1(precision, recall))

0.23376082300456397


In [26]:
precision = calculate_precision(TRAIN_DATA, predicted_results)
recall = calculate_recall(TRAIN_DATA, predicted_results)
f1_score = 2*(precision*recall)/(precision+recall)
print(f"F1 Score: {f1_score}")

Precision: 0.18608478934260148
Recall: 0.8965468750000001
F1 Score: 0.3082003637339536


### Step 3. Repeat the same process on test set
Create parsing tree

In [19]:
hanlp_test_file = f"data/hanlp_con_test_results.pkl"
if Path(hanlp_test_file).exists():
    with open(hanlp_test_file, "rb") as f:
        hanlp_test_results = pickle.load(f)
else:
    hanlp_test_results = [get_nps_hanlp(predictor, d) for d in TEST_DATA]
    with open(hanlp_test_file, "wb") as f:
        pickle.dump(hanlp_test_results, f)

Get pages via wiki online api

In [20]:
test_doc_path = f"data/test_doc5.jsonl"
test_doc_path_aicup = f"data/test_doc5_aicup.jsonl"
test_doc_path_search = f"data/test_doc5_search.jsonl"
test_doc_path_tfidf = f"data/test_doc5_tfidf.jsonl"

if Path(test_doc_path).exists():
    with open(test_doc_path, "r", encoding="utf8") as f:
        test_results = pd.Series([
            set(json.loads(line)["predicted_pages"])
            for line in f
        ])
else:
    if Path(test_doc_path_search).exists():
        with open(test_doc_path_search, "r", encoding="utf8") as f:
            test_results_search = pd.Series([
                set(json.loads(line)["predicted_pages"])
                for line in f
            ], name="search")
    else:
        pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)
        test_df = pd.DataFrame(TEST_DATA)
        test_df.loc[:, "hanlp_results"] = hanlp_test_results
        # predicted_results = test_df.progress_apply(get_pred_pages, axis=1)
        test_results_search = test_df.parallel_apply(
            get_pred_pages_search, axis=1)
        save_doc(TEST_DATA, test_results_search, mode="test", suffix="_search")

    if Path(test_doc_path_aicup).exists():
        with open(test_doc_path_aicup, "r", encoding="utf8") as f:
            test_results_aicup = pd.Series([
                set(json.loads(line)["predicted_pages"])
                for line in f
            ], name="aicup")
    else:
        pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)
        test_df = pd.DataFrame(TEST_DATA)
        test_df.loc[:, "hanlp_results"] = hanlp_test_results
        # predicted_results = test_df.progress_apply(get_pred_pages, axis=1)
        test_results_aicup = test_df.parallel_apply(
            get_pred_pages, axis=1)
        save_doc(TEST_DATA, test_results_aicup, mode="test", suffix="_aicup")

    if Path(test_doc_path_tfidf).exists():
        with open(test_doc_path_tfidf, "r", encoding="utf8") as f:
            test_results_tfidf = pd.Series([
                set(json.loads(line)["predicted_pages"])
                for line in f
            ], name="tfidf")
    else:
        pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=3)
        TEST_DATA_SEARCH = load_json(test_doc_path_search)
        test_df_search = pd.DataFrame(TEST_DATA_SEARCH)
        test_results_tfidf = test_df_search.parallel_apply(
            partial(
                get_pred_pages_tfidf,
                tokenizing_method=partial(tokenize, stopwords=stopwords),
                vectorizer=vectorizer,
                tf_idf_matrix=X,
                wiki_pages=wiki_pages,
                topk=topk,
            ), axis=1)
        save_doc(TEST_DATA, test_results_tfidf, mode="test", suffix="_tfidf")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.409 seconds.
Prefix dict has been built successfully.


['顯微鏡學', '顯微鏡', '數碼顯微鏡', '掃描探針顯微鏡', '肉眼']


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.369 seconds.
Prefix dict has been built successfully.


['蜜蜂屬', '巢蜜', '蜂蜜']
['綠山城縣', '伊沃瓦', '綠山城']
['百科全書派', '百科全書，或科學、藝術和工藝詳解詞典', '安·羅伯特·雅克·杜閣', '保羅派', '哈斯卡拉運動']
['魂斷藍橋', '亨格福德橋和五十週年紀念橋']


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


['京臺高速鐵路', '京九鐵路', '京廣鐵路', '慶全線', '京仁線']


Loading model cost 0.356 seconds.
Prefix dict has been built successfully.


['指南車']
['鼕鼕的假期', '侯孝賢', '泰迪熊獎', '刺客聶隱娘', '第27屆金馬獎']
['希臘神話', '神話學', '羅馬神話', '北歐神話']
[]
[]
[]
['2012年金星凌日']
['世界地質公園列表', '中國國家地質公園', '中國省級風景名勝區', '甘肅張掖國家地質公園', '中華人民共和國國家級風景名勝區']
['馬克思學說研究會', '馬克思主義思想流派', '新馬克思主義', '西方馬克思主義', '對馬克思主義的批評']
['蜜蜂屬', '蜂蜜']
['投資型硬幣', '銀']
['一貫道', '彌勒教', '漢傳佛教', '蓮花生', '慚愧祖師']
['索馬利蘭國旗', '索馬里國旗', '北馬其頓國旗', '馬拉維國旗', '北萊茵-威斯特法倫州旗']
[]
['總統制', '行政法', '政府首腦', '立法機關', '行政機關']
['歐幾里得空間']
['愛爾蘭國家的名稱', '愛爾蘭自由邦']
['新北市區公車790路線', '基隆捷運', '基隆車站', '臺中市公車131路', '新北市區公車575路線']
['回族', '伊斯蘭教的傳播', '穆斯林', '穆斯林世界', '伊斯蘭教']
['左鎮北極殿']
[]
['世界大學學術排名', '愛丁堡', '世界銀行', '愛丁堡納皮爾大學', '愛丁堡大學']
['撞擊坑', '普維斯撞擊坑', '撞擊結構', '鷹撞擊坑']
['希臘神話', '神', '北歐神話', '自然神論', '半神']
['煉武臺站', '江景線', '一山站', '炭峴站', '江景站']
[]
[]
['天鵝湖', '芭蕾舞劇列表']
['法輪功', '法輪功在臺灣', '法輪功在各地的發展及影響', '對法輪功的鎮壓']
['軍政府', '臺灣戰後時期', '軍事管治', '路易斯·伊納西奧·盧拉·達席爾瓦']
['阿里·本·阿比·塔利卜', '伊斯蘭教的先知', '穆罕默德', '伊斯蘭教', '聖經與古蘭經的相似之處']
[]
['奧斯卡', '奧斯卡最佳實景短片獎', '奧斯卡金像獎']
[]
['歐內斯特·勞倫斯', '馬克·奧利芬特', '羅伯特·奧本海默', '格倫·西奧多·西博格']
['地震波', '地震', '2011年日本東北地方太平洋近海地

In [23]:
results_df = pd.merge(pd.Series([line for line in test_results_aicup], name="aicup"), 
                      pd.Series([line for line in test_results_tfidf], name="tfidf"), right_index=True, left_index=True)
test_results = results_df.apply(union_result, axis=1)
save_doc(TEST_DATA, test_results, mode="test")

{'肉眼', '電磁學', '顯微鏡', '光學顯微鏡'} {'顯微鏡', '數碼顯微鏡', '掃描探針顯微鏡', '肉眼', '顯微鏡學'}
{'昆蟲', '蜜蜂屬', '花蜜', '蠶', '絲綢'} {'蜂蜜', '蜜蜂屬', '巢蜜'}
{'西部', '土地', '波蘭', '山城', '綠山城縣', '0', '綠山城'} {'綠山城', '綠山城縣', '伊沃瓦'}
{'橋', '配角', '藍橋', '魂斷藍橋'} {'亨格福德橋和五十週年紀念橋', '魂斷藍橋'}
{'金馬獎', '獎勵', '侯孝賢', '文學', '電影', '唐朝', '劇情片', '文言文'} {'侯孝賢', '第27屆金馬獎', '泰迪熊獎', '刺客聶隱娘', '鼕鼕的假期'}
{'薄熙來', '副總理', '姪', '國務院'} set()
{'凌日', '水星', '水星凌日'} {'2012年金星凌日'}
{'作品', '馬克思·多爾穆瓦站', '政治經濟學', '海因裏希·馬克思', '馬克思主義', '馬克思主義政治經濟學', '格魯喬·馬克思', '卡爾·馬克思'} {'馬克思學說研究會', '馬克思主義思想流派', '新馬克思主義', '西方馬克思主義', '對馬克思主義的批評'}
{'無生老母', '一貫道', '祖師', '路', '生', '母親', '彌勒菩薩', '神'} {'蓮花生', '一貫道', '慚愧祖師', '彌勒教', '漢傳佛教'}
{'伊斯蘭教', '回族', '內蒙古自治區', '穆斯林', '新疆維吾爾自治區', '中亞', '西亞'} {'穆斯林世界', '回族', '伊斯蘭教', '伊斯蘭教的傳播', '穆斯林'}
{'撞擊坑', '天衛四', '地形', '彗星', '行星', '面_(幾何)', '小行星'} {'撞擊結構', '撞擊坑', '鷹撞擊坑', '普維斯撞擊坑'}
{'玲子·艾爾斯沃斯', '身', '血親', '美洲', '亞洲'} set()
{'法輪功', '口耳相傳', '中國', '臺灣'} {'法輪功在各地的發展及影響', '法輪功', '對法輪功的鎮壓', '法輪功在臺灣'}
{'獎勵', '奧斯卡', '小說', '名', '奧斯卡·埃爾南德斯'} {'奧斯卡', '奧斯卡金像獎', '奧斯卡

notebook2
## PART 2. Sentence retrieval

Import some libs

In [1]:
from pathlib import Path
from typing import Dict, List, Set, Tuple, Union

# third-party libs
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_scheduler,
)

from dataset import BERTDataset, Dataset

# local libs
from utils import (
    generate_evidence_to_wiki_pages_mapping,
    jsonl_dir_to_df,
    load_json,
    load_model,
    save_checkpoint,
    set_lr_scheduler,
)

pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)

Global variable

In [2]:
SEED = 42

TRAIN_DATA = load_json("data/public_train.jsonl")
TEST_DATA = load_json("data/public_test.jsonl")
DOC_DATA = load_json("data/train_doc5.jsonl")

LABEL2ID: Dict[str, int] = {
    "supports": 0,
    "refutes": 1,
    "NOT ENOUGH INFO": 2,
}
ID2LABEL: Dict[int, str] = {v: k for k, v in LABEL2ID.items()}

_y = [LABEL2ID[data["label"]] for data in TRAIN_DATA]
# GT means Ground Truth
TRAIN_GT, DEV_GT = train_test_split(
    DOC_DATA,
    test_size=0.2,
    random_state=SEED,
    shuffle=True,
    stratify=_y,
)

Preload wiki database (1 min)

In [3]:
wiki_pages = jsonl_dir_to_df("data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages)
del wiki_pages

Reading and concatenating jsonl files in data/wiki-pages
Generate parse mapping


Transform to id to evidence_map mapping


### Helper function

Calculate precision for sentence retrieval

In [4]:
def evidence_macro_precision(
    instance: Dict,
    top_rows: pd.DataFrame,
) -> Tuple[float, float]:
    """Calculate precision for sentence retrieval
    This function is modified from fever-scorer.
    https://github.com/sheffieldnlp/fever-scorer/blob/master/src/fever/scorer.py

    Args:
        instance (dict): a row of the dev set (dev.jsonl) of test set (test.jsonl)
        top_rows (pd.DataFrame): our predictions with the top probabilities

        IMPORTANT!!!
        instance (dict) should have the key of `evidence`.
        top_rows (pd.DataFrame) should have a column `predicted_evidence`.

    Returns:
        Tuple[float, float]:
        [1]: relevant and retrieved (numerator of precision)
        [2]: retrieved (denominator of precision)
    """
    this_precision = 0.0
    this_precision_hits = 0.0

    # Return 0, 0 if label is not enough info since not enough info does not
    # contain any evidence.
    if instance["label"].upper() != "NOT ENOUGH INFO":
        # e[2] is the page title, e[3] is the sentence index
        all_evi = [[e[2], e[3]]
                   for eg in instance["evidence"]
                   for e in eg
                   if e[3] is not None]
        claim = instance["claim"]
        predicted_evidence = top_rows[top_rows["claim"] ==
                                      claim]["predicted_evidence"].tolist()

        for prediction in predicted_evidence:
            if prediction in all_evi:
                this_precision += 1.0
            this_precision_hits += 1.0

        return (this_precision /
                this_precision_hits) if this_precision_hits > 0 else 1.0, 1.0

    return 0.0, 0.0

Calculate recall for sentence retrieval

In [5]:
def evidence_macro_recall(
    instance: Dict,
    top_rows: pd.DataFrame,
) -> Tuple[float, float]:
    """Calculate recall for sentence retrieval
    This function is modified from fever-scorer.
    https://github.com/sheffieldnlp/fever-scorer/blob/master/src/fever/scorer.py

    Args:
        instance (dict): a row of the dev set (dev.jsonl) of test set (test.jsonl)
        top_rows (pd.DataFrame): our predictions with the top probabilities

        IMPORTANT!!!
        instance (dict) should have the key of `evidence`.
        top_rows (pd.DataFrame) should have a column `predicted_evidence`.

    Returns:
        Tuple[float, float]:
        [1]: relevant and retrieved (numerator of recall)
        [2]: relevant (denominator of recall)
    """
    # We only want to score F1/Precision/Recall of recalled evidence for NEI claims
    if instance["label"].upper() != "NOT ENOUGH INFO":
        # If there's no evidence to predict, return 1
        if len(instance["evidence"]) == 0 or all(
            [len(eg) == 0 for eg in instance]):
            return 1.0, 1.0

        claim = instance["claim"]

        predicted_evidence = top_rows[top_rows["claim"] ==
                                      claim]["predicted_evidence"].tolist()

        for evidence_group in instance["evidence"]:
            evidence = [[e[2], e[3]] for e in evidence_group]
            if all([item in predicted_evidence for item in evidence]):
                # We only want to score complete groups of evidence. Incomplete
                # groups are worthless.
                return 1.0, 1.0
        return 0.0, 1.0
    return 0.0, 0.0

Calculate the scores of sentence retrieval

In [6]:
def evaluate_retrieval(
    probs: np.ndarray,
    df_evidences: pd.DataFrame,
    ground_truths: pd.DataFrame,
    top_n: int = 5,
    cal_scores: bool = True,
    save_name: str = None,
) -> Dict[str, float]:
    """Calculate the scores of sentence retrieval

    Args:
        probs (np.ndarray): probabilities of the candidate retrieved sentences
        df_evidences (pd.DataFrame): the candiate evidence sentences paired with claims
        ground_truths (pd.DataFrame): the loaded data of dev.jsonl or test.jsonl
        top_n (int, optional): the number of the retrieved sentences. Defaults to 2.

    Returns:
        Dict[str, float]: F1 score, precision, and recall
    """
    df_evidences["prob"] = probs
    top_rows = (
        df_evidences.groupby("claim").apply(
        lambda x: x.nlargest(top_n, "prob"))
        .reset_index(drop=True)
    )

    if cal_scores:
        macro_precision = 0
        macro_precision_hits = 0
        macro_recall = 0
        macro_recall_hits = 0

        for i, instance in enumerate(ground_truths):
            macro_prec = evidence_macro_precision(instance, top_rows)
            macro_precision += macro_prec[0]
            macro_precision_hits += macro_prec[1]

            macro_rec = evidence_macro_recall(instance, top_rows)
            macro_recall += macro_rec[0]
            macro_recall_hits += macro_rec[1]

        pr = (macro_precision /
              macro_precision_hits) if macro_precision_hits > 0 else 1.0
        rec = (macro_recall /
               macro_recall_hits) if macro_recall_hits > 0 else 0.0
        f1 = 2.0 * pr * rec / (pr + rec)

    if save_name is not None:
        # write doc7_sent5 file
        with open(f"data/{save_name}", "w", encoding="utf8") as f:
            for instance in ground_truths:
                claim = instance["claim"]
                predicted_evidence = top_rows[
                    top_rows["claim"] == claim]["predicted_evidence"].tolist()
                instance["predicted_evidence"] = predicted_evidence
                f.write(json.dumps(instance, ensure_ascii=False) + "\n")

    if cal_scores:
        return {"F1 score": f1, "Precision": pr, "Recall": rec}

Inference script to get probabilites for the candidate evidence sentences

In [7]:
def get_predicted_probs(
    model: nn.Module,
    dataloader: Dataset,
    device: torch.device,
) -> np.ndarray:
    """Inference script to get probabilites for the candidate evidence sentences

    Args:
        model: the one from HuggingFace Transformers
        dataloader: devset or testset in torch dataloader

    Returns:
        np.ndarray: probabilites of the candidate evidence sentences
    """
    model.eval()
    probs = []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            probs.extend(torch.softmax(logits, dim=1)[:, 1].tolist())

    return np.array(probs)

AicupTopkEvidenceBERTDataset class for AICUP dataset with top-k evidence sentences

In [8]:
class SentRetrievalBERTDataset(BERTDataset):
    """AicupTopkEvidenceBERTDataset class for AICUP dataset with top-k evidence sentences."""

    def __getitem__(
        self,
        idx: int,
        **kwargs,
    ) -> Tuple[Dict[str, torch.Tensor], int]:
        item = self.data.iloc[idx]
        sentA = item["claim"]
        sentB = item["text"]

        # claim [SEP] text
        concat = self.tokenizer(
            sentA,
            sentB,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
        )
        concat_ten = {k: torch.tensor(v) for k, v in concat.items()}
        if "label" in item:
            concat_ten["labels"] = torch.tensor(item["label"])

        return concat_ten

### Main function for sentence retrieval

In [9]:
def pair_with_wiki_sentences(
    mapping: Dict[str, Dict[int, str]],
    df: pd.DataFrame,
    negative_ratio: float,
) -> pd.DataFrame:
    """Only for creating train sentences."""
    claims = []
    sentences = []
    labels = []

    # positive
    for i in range(len(df)):
        if df["label"].iloc[i] == "NOT ENOUGH INFO":
            continue

        claim = df["claim"].iloc[i]
        evidence_sets = df["evidence"].iloc[i]
        for evidence_set in evidence_sets:
            sents = []
            for evidence in evidence_set:
                # evidence[2] is the page title
                page = evidence[2].replace(" ", "_")
                # the only page with weird name
                if page == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
                    continue
                # evidence[3] is in form of int however, mapping requires str
                sent_idx = str(evidence[3])
                sents.append(mapping[page][sent_idx])

            whole_evidence = " ".join(sents)

            claims.append(claim)
            sentences.append(whole_evidence)
            labels.append(1)

    # negative
    for i in range(len(df)):
        if df["label"].iloc[i] == "NOT ENOUGH INFO":
            continue
        claim = df["claim"].iloc[i]

        evidence_set = set([(evidence[2], evidence[3])
                            for evidences in df["evidence"][i]
                            for evidence in evidences])
        predicted_pages = df["predicted_pages"][i]
        for page in predicted_pages:
            page = page.replace(" ", "_")
            try:
                page_sent_id_pairs = [
                    (page, sent_idx) for sent_idx in mapping[page].keys()
                ]
            except KeyError:
                # print(f"{page} is not in our Wiki db.")
                continue

            for pair in page_sent_id_pairs:
                if pair in evidence_set:
                    continue
                text = mapping[page][pair[1]]
                # `np.random.rand(1) <= 0.05`: Control not to add too many negative samples
                if text != "" and np.random.rand(1) <= negative_ratio:
                    claims.append(claim)
                    sentences.append(text)
                    labels.append(0)

    return pd.DataFrame({"claim": claims, "text": sentences, "label": labels})


def pair_with_wiki_sentences_eval(
    mapping: Dict[str, Dict[int, str]],
    df: pd.DataFrame,
    is_testset: bool = False,
) -> pd.DataFrame:
    """Only for creating dev and test sentences."""
    claims = []
    sentences = []
    evidence = []
    predicted_evidence = []

    # negative
    for i in range(len(df)):
        # if df["label"].iloc[i] == "NOT ENOUGH INFO":
        #     continue
        claim = df["claim"].iloc[i]

        predicted_pages = df["predicted_pages"][i]
        for page in predicted_pages:
            page = page.replace(" ", "_")
            try:
                page_sent_id_pairs = [(page, k) for k in mapping[page]]
            except KeyError:
                # print(f"{page} is not in our Wiki db.")
                continue

            for page_name, sentence_id in page_sent_id_pairs:
                text = mapping[page][sentence_id]
                if text != "":
                    claims.append(claim)
                    sentences.append(text)
                    if not is_testset:
                        evidence.append(df["evidence"].iloc[i])
                    predicted_evidence.append([page_name, int(sentence_id)])

    return pd.DataFrame({
        "claim": claims,
        "text": sentences,
        "evidence": evidence if not is_testset else None,
        "predicted_evidence": predicted_evidence,
    })

### Step 1. Setup training environment

Hyperparams

In [31]:
MODEL_NAME = "bert-base-chinese"  #@param {type:"string"}
NUM_EPOCHS = 1  #@param {type:"integer"}
LR = 2e-5  #@param {type:"number"}
TRAIN_BATCH_SIZE = 32  #@param {type:"integer"}
TEST_BATCH_SIZE = 256  #@param {type:"integer"}
NEGATIVE_RATIO = 0.05  #@param {type:"number"}
VALIDATION_STEP = 50  #@param {type:"integer"}
TOP_N = 5  #@param {type:"integer"}
#@title  { display-mode: "form" }

Experiment Directory

In [32]:
EXP_DIR = f"sent_retrieval/e{NUM_EPOCHS}_bs{TRAIN_BATCH_SIZE}_" + f"{LR}_neg{NEGATIVE_RATIO}_top{TOP_N}"
LOG_DIR = "logs/" + EXP_DIR
CKPT_DIR = "checkpoints/" + EXP_DIR

if not Path(LOG_DIR).exists():
    Path(LOG_DIR).mkdir(parents=True)

if not Path(CKPT_DIR).exists():
    Path(CKPT_DIR).mkdir(parents=True)

### Step 2. Combine claims and evidences

In [33]:
train_df = pair_with_wiki_sentences(
    mapping,
    pd.DataFrame(TRAIN_GT),
    NEGATIVE_RATIO,
)
counts = train_df["label"].value_counts()
print("Now using the following train data with 0 (Negative) and 1 (Positive)")
print(counts)

dev_evidences = pair_with_wiki_sentences_eval(mapping, pd.DataFrame(DEV_GT))

Now using the following train data with 0 (Negative) and 1 (Positive)
0    7933
1    2774
Name: label, dtype: int64


### Step 3. Start training

Dataloader things

In [34]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = SentRetrievalBERTDataset(train_df, tokenizer=tokenizer)
val_dataset = SentRetrievalBERTDataset(dev_evidences, tokenizer=tokenizer)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
)
eval_dataloader = DataLoader(val_dataset, batch_size=TEST_BATCH_SIZE)

Save your memory.

In [2]:
del train_df

NameError: name 'train_df' is not defined

Trainer

In [36]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
print(torch.cuda.is_available())
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.to(device)

optimizer = AdamW(model.parameters(), lr=LR)
num_training_steps = NUM_EPOCHS * len(train_dataloader)
lr_scheduler = set_lr_scheduler(optimizer, num_training_steps)

writer = SummaryWriter(LOG_DIR)

True


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Please make sure that you are using gpu when training (5 min)

In [28]:
torch.cuda.empty_cache()

In [18]:
progress_bar = tqdm(range(num_training_steps))
current_steps = 0

for epoch in range(NUM_EPOCHS):
    model.train()

    for batch in train_dataloader:
        torch.cuda.empty_cache()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        writer.add_scalar("training_loss", loss.item(), current_steps)

        y_pred = torch.argmax(outputs.logits, dim=1).tolist()
        y_true = batch["labels"].tolist()

        current_steps += 1

        if current_steps % VALIDATION_STEP == 0 and current_steps > 0:
            print("Start validation")
            probs = get_predicted_probs(model, eval_dataloader, device)

            val_results = evaluate_retrieval(
                probs=probs,
                df_evidences=dev_evidences,
                ground_truths=DEV_GT,
                top_n=TOP_N,
            )
            print(val_results)

            # log each metric separately to TensorBoard
            for metric_name, metric_value in val_results.items():
                writer.add_scalar(
                    f"dev_{metric_name}",
                    metric_value,
                    current_steps,
                )

            save_checkpoint(model, CKPT_DIR, current_steps)

print("Finished training!")

  0%|          | 0/136 [00:00<?, ?it/s]

Start validation


  0%|          | 0/189 [00:00<?, ?it/s]

{'F1 score': 0.38156865612499047, 'Precision': 0.2522656249999999, 'Recall': 0.7828125}
Start validation


  0%|          | 0/189 [00:00<?, ?it/s]

{'F1 score': 0.3842702317557939, 'Precision': 0.25414062499999995, 'Recall': 0.7875}
Finished training!


In [34]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 17672), started 0:15:52 ago. (Use '!kill 17672' to kill it.)

Validation part (15 mins)

In [37]:
torch.cuda.empty_cache()

In [38]:
import json
ckpt_name = "model.200.pt"  #@param {type:"string"}
model = load_model(model, ckpt_name, CKPT_DIR)
print("Start final evaluations and write prediction files.")

train_evidences = pair_with_wiki_sentences_eval(
    mapping=mapping,
    df=pd.DataFrame(TRAIN_GT),
)
train_set = SentRetrievalBERTDataset(train_evidences, tokenizer)
train_dataloader = DataLoader(train_set, batch_size=TEST_BATCH_SIZE)

print("Start calculating training scores")
probs = get_predicted_probs(model, train_dataloader, device)
train_results = evaluate_retrieval(
    probs=probs,
    df_evidences=train_evidences,
    ground_truths=TRAIN_GT,
    top_n=TOP_N,
    save_name=f"train_doc5sent{TOP_N}.jsonl",
)
print(f"Training scores => {train_results}")

print("Start validation")
probs = get_predicted_probs(model, eval_dataloader, device)
val_results = evaluate_retrieval(
    probs=probs,
    df_evidences=dev_evidences,
    ground_truths=DEV_GT,
    top_n=TOP_N,
    save_name=f"dev_doc5sent{TOP_N}.jsonl",
)

print(f"Validation scores => {val_results}")

Start final evaluations and write prediction files.
Start calculating training scores


  0%|          | 0/748 [00:00<?, ?it/s]

Training scores => {'F1 score': 0.4020991641337401, 'Precision': 0.26822916666666796, 'Recall': 0.802734375}
Start validation


  0%|          | 0/189 [00:00<?, ?it/s]

Validation scores => {'F1 score': 0.38408393111995776, 'Precision': 0.25414062499999984, 'Recall': 0.7859375}


Load the model we want.

In [29]:
ckpt_name = "model.200.pt"
model = load_model(model, ckpt_name, CKPT_DIR)

### Step 4. Check on our test data
(5 min)

In [30]:
test_data = load_json("data/test_doc5.jsonl")

test_evidences = pair_with_wiki_sentences_eval(
    mapping,
    pd.DataFrame(test_data),
    is_testset=True,
)
test_set = SentRetrievalBERTDataset(test_evidences, tokenizer)
test_dataloader = DataLoader(test_set, batch_size=TEST_BATCH_SIZE)

print("Start predicting the test data")
probs = get_predicted_probs(model, test_dataloader, device)
evaluate_retrieval(
    probs=probs,
    df_evidences=test_evidences,
    ground_truths=test_data,
    top_n=TOP_N,
    cal_scores=False,
    save_name=f"test_doc5sent{TOP_N}.jsonl",
)

Start predicting the test data


  0%|          | 0/259 [00:00<?, ?it/s]

In [ ]:
del wiki_pages

notebook3
## PART 3. Claim verification

import libs

In [1]:
import pickle
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
from pandarallel import pandarallel
from tqdm.auto import tqdm

import torch
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_scheduler,
)

from dataset import BERTDataset
from utils import (
    generate_evidence_to_wiki_pages_mapping,
    jsonl_dir_to_df,
    load_json,
    load_model,
    save_checkpoint,
    set_lr_scheduler,
)

pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=4)

Global variables

In [2]:
LABEL2ID: Dict[str, int] = {
    "supports": 0,
    "refutes": 1,
    "NOT ENOUGH INFO": 2,
}
ID2LABEL: Dict[int, str] = {v: k for k, v in LABEL2ID.items()}

TRAIN_DATA = load_json("data/train_doc5sent5.jsonl")
DEV_DATA = load_json("data/dev_doc5sent5.jsonl")

TRAIN_PKL_FILE = Path("data/train_doc5sent5.pkl")
DEV_PKL_FILE = Path("data/dev_doc5sent5.pkl")

Preload wiki database (same as part 2.)

In [3]:
wiki_pages = jsonl_dir_to_df("data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages,)
del wiki_pages

Reading and concatenating jsonl files in data/wiki-pages
Generate parse mapping


Transform to id to evidence_map mapping


### Helper function

AICUP dataset with top-k evidence sentences.

In [4]:
class AicupTopkEvidenceBERTDataset(BERTDataset):
    """AICUP dataset with top-k evidence sentences."""

    def __getitem__(
        self,
        idx: int,
        **kwargs,
    ) -> Tuple[Dict[str, torch.Tensor], int]:
        item = self.data.iloc[idx]
        claim = item["claim"]
        evidence = item["evidence_list"]

        # In case there are less than topk evidence sentences
        pad = ["[PAD]"] * (self.topk - len(evidence))
        evidence += pad
        concat_claim_evidence = " [SEP] ".join([*claim, *evidence])

        concat = self.tokenizer(
            concat_claim_evidence,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
        )
        label = LABEL2ID[item["label"]] if "label" in item else -1
        concat_ten = {k: torch.tensor(v) for k, v in concat.items()}

        if "label" in item:
            concat_ten["labels"] = torch.tensor(label)

        return concat_ten

Evaluation function

In [5]:
def run_evaluation(model: torch.nn.Module, dataloader: DataLoader, device):
    model.eval()

    loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            y_true.extend(batch["labels"].tolist())

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss += outputs.loss.item()
            logits = outputs.logits
            y_pred.extend(torch.argmax(logits, dim=1).tolist())

    acc = accuracy_score(y_true, y_pred)

    return {"val_loss": loss / len(dataloader), "val_acc": acc}

Prediction

In [6]:
def run_predict(model: torch.nn.Module, test_dl: DataLoader, device) -> list:
    model.eval()

    preds = []
    for batch in tqdm(test_dl,
                      total=len(test_dl),
                      leave=False,
                      desc="Predicting"):
        batch = {k: v.to(device) for k, v in batch.items()}
        pred = model(**batch).logits
        pred = torch.argmax(pred, dim=1)
        preds.extend(pred.tolist())
    return preds

### Main function

In [7]:
def join_with_topk_evidence(
    df: pd.DataFrame,
    mapping: dict,
    mode: str = "train",
    topk: int = 5,
) -> pd.DataFrame:
    """join_with_topk_evidence join the dataset with topk evidence.

    Note:
        After extraction, the dataset will be like this:
               id     label         claim                           evidence            evidence_list
        0    4604  supports       高行健...     [[[3393, 3552, 高行健, 0], [...  [高行健 （ ）江西赣州出...
        ..    ...       ...            ...                                ...                     ...
        945  2095  supports       美國總...  [[[1879, 2032, 吉米·卡特, 16], [...  [卸任后 ， 卡特積極參與...
        停各种战争及人質危機的斡旋工作 ， 反对美国小布什政府攻打伊拉克...

        [946 rows x 5 columns]

    Args:
        df (pd.DataFrame): The dataset with evidence.
        wiki_pages (pd.DataFrame): The wiki pages dataframe
        topk (int, optional): The topk evidence. Defaults to 5.
        cache(Union[Path, str], optional): The cache file path. Defaults to None.
            If cache is None, return the result directly.

    Returns:
        pd.DataFrame: The dataset with topk evidence_list.
            The `evidence_list` column will be: List[str]
    """

    # format evidence column to List[List[Tuple[str, str, str, str]]]
    if "evidence" in df.columns:
        df["evidence"] = df["evidence"].parallel_map(
            lambda x: [[x]] if not isinstance(x[0], list) else [x]
            if not isinstance(x[0][0], list) else x)

    print(f"Extracting evidence_list for the {mode} mode ...")
    if mode == "eval":
        # extract evidence
        df["evidence_list"] = df["predicted_evidence"].parallel_map(lambda x: [
            mapping.get(evi_id, {}).get(str(evi_idx), "")
            for evi_id, evi_idx in x  # for each evidence list
        ][:topk] if isinstance(x, list) else [])
        print(df["evidence_list"][:5])
    else:
        # extract evidence
        df["evidence_list"] = df["evidence"].parallel_map(lambda x: [
            " ".join([  # join evidence
                mapping.get(evi_id, {}).get(str(evi_idx), "")
                for _, _, evi_id, evi_idx in evi_list
            ]) if isinstance(evi_list, list) else ""
            for evi_list in x  # for each evidence list
        ][:1] if isinstance(x, list) else [])

    return df

### Step 1. Setup training environment

Hyperparams

In [8]:
#@title  { display-mode: "form" }

MODEL_NAME = "bert-base-chinese"  #@param {type:"string"}
TRAIN_BATCH_SIZE = 16  #@param {type:"integer"}
TEST_BATCH_SIZE = 32  #@param {type:"integer"}
SEED = 42  #@param {type:"integer"}
LR = 7e-5  #@param {type:"number"}
NUM_EPOCHS = 20  #@param {type:"integer"}
MAX_SEQ_LEN = 256  #@param {type:"integer"}
EVIDENCE_TOPK = 5  #@param {type:"integer"}
VALIDATION_STEP = 25  #@param {type:"integer"}


Experiment Directory

In [9]:
OUTPUT_FILENAME = "submission.jsonl"

EXP_DIR = f"claim_verification/e{NUM_EPOCHS}_bs{TRAIN_BATCH_SIZE}_" + f"{LR}_top{EVIDENCE_TOPK}"
LOG_DIR = "logs/" + EXP_DIR
CKPT_DIR = "checkpoints/" + EXP_DIR

if not Path(LOG_DIR).exists():
    Path(LOG_DIR).mkdir(parents=True)

if not Path(CKPT_DIR).exists():
    Path(CKPT_DIR).mkdir(parents=True)

### Step 2. Concat claim and evidences
join topk evidence

In [15]:
if not TRAIN_PKL_FILE.exists():
    train_df = join_with_topk_evidence(
        pd.DataFrame(TRAIN_DATA),
        mapping,
        topk=EVIDENCE_TOPK,
    )
    train_df.to_pickle(TRAIN_PKL_FILE, protocol=4)
else:
    with open(TRAIN_PKL_FILE, "rb") as f:
        train_df = pickle.load(f)

if not DEV_PKL_FILE.exists():
    dev_df = join_with_topk_evidence(
        pd.DataFrame(DEV_DATA),
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )
    dev_df.to_pickle(DEV_PKL_FILE, protocol=4)
else:
    with open(DEV_PKL_FILE, "rb") as f:
        dev_df = pickle.load(f)

Extracting evidence_list for the train mode ...


### Step 3. Training

Prevent CUDA out of memory

In [16]:
torch.cuda.empty_cache()

In [17]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = AicupTopkEvidenceBERTDataset(
    train_df,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
)
val_dataset = AicupTopkEvidenceBERTDataset(
    dev_df,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=0,
)
eval_dataloader = DataLoader(val_dataset, batch_size=TEST_BATCH_SIZE, num_workers=0,)

In [18]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cpu")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABEL2ID),
)
torch.cuda.empty_cache()
model.to(device)
optimizer = AdamW(model.parameters(), lr=LR)
num_training_steps = NUM_EPOCHS * len(train_dataloader)
lr_scheduler = set_lr_scheduler(optimizer, num_training_steps)

writer = SummaryWriter(LOG_DIR)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training (30 mins)

In [113]:
progress_bar = tqdm(range(num_training_steps))
current_steps = 0

for epoch in range(NUM_EPOCHS):
    model.train()

    for batch in train_dataloader:
        torch.cuda.empty_cache()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        writer.add_scalar("training_loss", loss.item(), current_steps)

        y_pred = torch.argmax(outputs.logits, dim=1).tolist()
        y_true = batch["labels"].tolist()

        current_steps += 1

        if current_steps % VALIDATION_STEP == 0 and current_steps > 0:
            print(f"Start validation: current_steps={current_steps}, epoch={epoch}")
            val_results = run_evaluation(model, eval_dataloader, device)

            # log each metric separately to TensorBoard
            for metric_name, metric_value in val_results.items():
                print(f"{metric_name}: {metric_value}")
                writer.add_scalar(f"{metric_name}", metric_value, current_steps)

            val_acc = val_results['val_acc']
            if val_acc > 0.4:
                save_checkpoint(
                    model,
                    CKPT_DIR,
                    current_steps,
                    mark=f"val_acc={val_acc:.4f}",
                )

print("Finished training!")

  0%|          | 0/3960 [00:00<?, ?it/s]

Start validation: current_steps=25, epoch=0


  0%|          | 0/25 [00:00<?, ?it/s]

val_loss: 1.0586568212509155
val_acc: 0.4005069708491762
Start validation: current_steps=50, epoch=0


  0%|          | 0/25 [00:00<?, ?it/s]

...

val_loss: 1.0510486102104186
val_acc: 0.4005069708491762
Start validation: current_steps=3925, epoch=19


  0%|          | 0/25 [00:00<?, ?it/s]

val_loss: 1.0510486102104186
val_acc: 0.4005069708491762
Start validation: current_steps=3950, epoch=19


  0%|          | 0/25 [00:00<?, ?it/s]

val_loss: 1.0510486102104186
val_acc: 0.4005069708491762
Finished training!


### Step 4. Make your submission

In [19]:
TEST_DATA = load_json("data/test_doc5sent5.jsonl")
TEST_PKL_FILE = Path("data/test_doc5sent5.pkl")

if not TEST_PKL_FILE.exists():
    test_df = join_with_topk_evidence(
        pd.DataFrame(TEST_DATA),
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )
    test_df.to_pickle(TEST_PKL_FILE, protocol=4)
else:
    with open(TEST_PKL_FILE, "rb") as f:
        test_df = pickle.load(f)

test_dataset = AicupTopkEvidenceBERTDataset(
    test_df,
    tokenizer=tokenizer,
    max_length=MAX_SEQ_LEN,
)
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE)

Prediction

In [118]:
torch.cuda.empty_cache()

In [119]:
ckpt_name = "val_acc=0.4056_model.1000.pt"  #@param {type:"string"}
model = load_model(model, ckpt_name, CKPT_DIR)
predicted_label = run_predict(model, test_dataloader, device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 7.93 GiB total capacity; 7.16 GiB already allocated; 19.19 MiB free; 7.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Write files

In [18]:
predict_dataset = test_df.copy()
predict_dataset["predicted_label"] = list(map(ID2LABEL.get, predicted_label))
predict_dataset[["id", "predicted_label", "predicted_evidence"]].to_json(
    f"submission/{ckpt_name[:14]}_{OUTPUT_FILENAME}",
    orient="records",
    lines=True,
    force_ascii=False,
)